# Gender Changing Levels

In [1]:
import pandas as pd
import pandas_ta as ta
df1 = pd.read_csv("data/monthly_segments/CL_30m_2025-03.csv")
df2 = pd.read_csv("data/monthly_segments/CL_30m_2025-04.csv")
df3 = pd.read_csv("data/monthly_segments/CL_30m_2025-05.csv")

df = pd.concat([df1, df2, df3])
df.columns=['datetime', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df['atr14'] = ta.atr(df['high'], df['low'], df['close'], length=14)
df.head(10)

,datetime,open,high,low,close,volume,atr14
0,2025-03-02 18:00:00,69.95,70.07,69.86,70.07,199,NaN
1,2025-03-02 18:30:00,70.08,70.24,70.00,70.21,1321,NaN
2,2025-03-02 19:00:00,70.21,70.23,70.15,70.20,644,NaN
3,2025-03-02 19:30:00,70.21,70.26,70.17,70.21,684,NaN
4,2025-03-02 20:00:00,70.21,70.38,70.21,70.34,1195,NaN
5,2025-03-02 20:30:00,70.34,70.52,70.32,70.49,2019,NaN
6,2025-03-02 21:00:00,70.49,70.50,70.40,70.50,853,NaN
7,2025-03-02 21:30:00,70.51,70.60,70.47,70.52,1226,NaN
8,2025-03-02 22:00:00,70.51,70.51,70.27,70.30,1069,NaN
9,2025-03-02 22:30:00,70.30,70.30,70.04,70.11,1663,NaN


In [2]:
right_peek = 10
left_peek = 10

def pivotid(df1, index, left_peek, right_peek): #left_peek right_peek before and after candle at index. 
    if index-left_peek < 0 or index+right_peek >= len(df1): # Avoid out of bounds. 
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(index-left_peek, index+right_peek+1):
        if(df1.low[index]>df1.low[i]):
            pividlow=0 # Check if current index is lowest inside of window. 
        if(df1.high[index]<df1.high[i]):
            pividhigh=0 # Check if current index is highest inside of window.

    if pividlow and pividhigh: # Edge case: Both swing high and low. 
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

# x.name is the numerical index, since we reset_index before. 
df[f'pivot'] = df.apply(lambda x: pivotid(df, x.name, right_peek, left_peek), axis=1)
df.head(10)


,datetime,open,high,low,close,volume,atr14,pivot
0,2025-03-02 18:00:00,69.95,70.07,69.86,70.07,199,NaN,0
1,2025-03-02 18:30:00,70.08,70.24,70.00,70.21,1321,NaN,0
2,2025-03-02 19:00:00,70.21,70.23,70.15,70.20,644,NaN,0
3,2025-03-02 19:30:00,70.21,70.26,70.17,70.21,684,NaN,0
4,2025-03-02 20:00:00,70.21,70.38,70.21,70.34,1195,NaN,0
5,2025-03-02 20:30:00,70.34,70.52,70.32,70.49,2019,NaN,0
6,2025-03-02 21:00:00,70.49,70.50,70.40,70.50,853,NaN,0
7,2025-03-02 21:30:00,70.51,70.60,70.47,70.52,1226,NaN,0
8,2025-03-02 22:00:00,70.51,70.51,70.27,70.30,1069,NaN,0
9,2025-03-02 22:30:00,70.30,70.30,70.04,70.11,1663,NaN,0


In [3]:
import numpy as np

def pointpos(x, distance):
    if x[f'pivot']==1:
        return x['low']-distance
    elif x[f'pivot']==2:
        return x['high']+distance
    else:
        return np.nan


# Point position for help plotting. 
df['pointpos'] = df.apply(lambda row: pointpos(row, distance=0.2), axis=1)
df.tail(10)

,datetime,open,high,low,close,volume,atr14,pivot,pointpos
2993,2025-05-30 12:30:00,60.32,60.42,59.74,59.98,10725,0.371039,0,NaN
2994,2025-05-30 13:00:00,59.97,60.48,59.91,60.40,8720,0.385250,0,NaN
2995,2025-05-30 13:30:00,60.39,60.66,60.29,60.58,8097,0.384161,0,NaN
2996,2025-05-30 14:00:00,60.58,60.81,60.50,60.65,9702,0.378864,0,NaN
2997,2025-05-30 14:30:00,60.66,60.97,60.59,60.71,12360,0.378945,0,NaN
2998,2025-05-30 15:00:00,60.71,60.78,60.61,60.75,4217,0.364020,0,NaN
2999,2025-05-30 15:30:00,60.75,60.78,60.61,60.75,2210,0.350162,0,NaN
3000,2025-05-30 16:00:00,60.75,60.90,60.74,60.89,2741,0.336579,0,NaN
3001,2025-05-30 16:30:00,60.88,60.93,60.82,60.84,1173,0.320394,0,NaN
3002,2025-05-30 17:00:00,60.85,60.85,60.72,60.79,1168,0.306795,0,NaN


In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                increasing_line_color= 'green', 
                decreasing_line_color= 'red')])

fig.add_scatter(x=df.index, y=df['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_layout(paper_bgcolor='black', plot_bgcolor='black')

fig.show()

In [5]:
import numpy as np

# Count how often pivot highs/lows land in the same price zone (>=2 hits become bands)
bin_width = 0.10  # 10 pips-wide zones
compression_distance = 0.40
MIN_TOUCHES = 5

support_slice = df.loc[df['pivot'] == 1, ['low', 'datetime']].dropna()
support_prices = support_slice['low']
support_dates = support_slice['datetime']

resistance_slice = df.loc[df['pivot'] == 2, ['high', 'datetime']].dropna()
resistance_prices = resistance_slice['high']
resistance_dates = resistance_slice['datetime']

support_levels = (support_prices / bin_width).round() * bin_width
resistance_levels = (resistance_prices / bin_width).round() * bin_width

level_summary = (
    pd.concat([
        pd.DataFrame({'level': support_levels, 'support_hits': 1, 'resistance_hits': 0, 'created': support_dates}),
        pd.DataFrame({'level': resistance_levels, 'support_hits': 0, 'resistance_hits': 1, 'created': resistance_dates})
    ])
    .groupby('level', as_index=False)
    .agg({'support_hits': 'sum', 'resistance_hits': 'sum', 'created': 'min'})
)
level_summary['total_hits'] = level_summary['support_hits'] + level_summary['resistance_hits']
level_summary = level_summary[level_summary['total_hits'] >= MIN_TOUCHES].sort_values('total_hits', ascending=False)
level_summary['type'] = np.where(level_summary['support_hits'] >= level_summary['resistance_hits'], 'support', 'resistance')

# Merge consecutive levels that lie within one bin width so only the stronger zone remains.
merged_levels = []
level_summary_sorted = level_summary.sort_values('level').reset_index(drop=True)
idx = 0
while idx < len(level_summary_sorted):
    current = level_summary_sorted.iloc[idx]
    if idx < len(level_summary_sorted) - 1:
        nxt = level_summary_sorted.iloc[idx + 1]
        if abs(current['level'] - nxt['level']) <= compression_distance:
            # Keep whichever level accumulated more total touches, and add the other's count.
            if current['total_hits'] >= nxt['total_hits']:
                merged = current.copy()
                merged['total_hits'] += nxt['total_hits']
                merged['support_hits'] += nxt['support_hits']
                merged['resistance_hits'] += nxt['resistance_hits']
            else:
                merged = nxt.copy()
                merged['total_hits'] += current['total_hits']
                merged['support_hits'] += current['support_hits']
                merged['resistance_hits'] += current['resistance_hits']
            # Recompute type after merging and set created time as earliest of the two.
            merged['type'] = 'support' if merged['support_hits'] >= merged['resistance_hits'] else 'resistance'
            merged['created'] = min(current['created'], nxt['created'])
            merged_levels.append(merged)
            idx += 2
            continue
    merged_levels.append(current)
    idx += 1

level_summary_merged = pd.DataFrame(merged_levels).sort_values('total_hits', ascending=False).reset_index(drop=True)
level_summary_merged.head()


,level,support_hits,resistance_hits,created,total_hits,type
0,61.5,6,5,2025-04-08 09:30:00,11,support
1,61.8,2,4,2025-04-07 23:00:00,6,resistance
2,67.5,2,3,2025-03-04 22:30:00,5,resistance


In [6]:
import plotly.graph_objects as go

# Highlight zones where we have at least two combined hits
zones = level_summary[level_summary['total_hits'] >= MIN_TOUCHES].copy()
zones['created'] = pd.to_datetime(zones['created'])

if zones.empty:
    print('No multi-hit zones found')
else:
    df_band = df.copy()
    df_band['datetime'] = pd.to_datetime(df_band['datetime'])

    fig = go.Figure(data=[go.Candlestick(
        x=df_band['datetime'],
        open=df_band['open'],
        high=df_band['high'],
        low=df_band['low'],
        close=df_band['close'],
        increasing_line_color='green',
        decreasing_line_color='red'
    )])

    half_width = bin_width / 2
    for _, zone in zones.iterrows():
        is_support = zone['support_hits'] > zone['resistance_hits']
        fill = 'rgba(72, 209, 204, 0.25)' if is_support else 'rgba(255, 0, 0, 0.25)'
        outline = 'rgba(72, 209, 204, 0.8)' if is_support else 'rgba(255, 0, 0, 0.8)'
        fig.add_shape(
            type='rect',
            xref='x',
            yref='y',
            x0=zone['created'],
            x1=df_band['datetime'].iloc[-1],
            y0=zone['level'] - half_width,
            y1=zone['level'] + half_width,
            line=dict(color=outline, width=1),
            fillcolor=fill
        )

    fig.update_layout(
        xaxis_rangeslider_visible=False,
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        paper_bgcolor='black',
        plot_bgcolor='black',
        title=f'Support/Resistance Bands (>={MIN_TOUCHES} hits)'
    )

    fig.show()


In [7]:
level_segments = []
flip_markers = []
threshold_mult = 2.0

df_levels = df.copy()
df_levels['datetime'] = pd.to_datetime(df_levels['datetime'])
zones_eval = level_summary_merged.copy()
zones_eval['created'] = pd.to_datetime(zones_eval['created'])
end_time = df_levels['datetime'].iloc[-1]

def add_segment(level, start, end, level_type, new_type=None):
    if end <= start:
        return
    level_segments.append({
        'level': level,
        'start': start,
        'flip_time': end,
        'type': level_type,
        'new_type': new_type
    })

for _, zone in zones_eval.iterrows():
    current_type = zone['type']
    level_price = zone['level']
    current_start = zone['created']
    window = df_levels[df_levels['datetime'] >= current_start]

    if window.empty:
        continue

    for _, candle in window.iterrows():
        atr_value = candle['atr14']
        if pd.isna(atr_value):
            continue
        threshold = atr_value * threshold_mult

        if current_type == 'resistance':
            if candle['high'] >= level_price + threshold:
                add_segment(level_price, current_start, candle['datetime'], current_type, 'support')
                flip_markers.append({'level': level_price, 'time': candle['datetime'], 'new_type': 'support'})
                current_type = 'support'
                current_start = candle['datetime']
        else:
            if candle['low'] <= level_price - threshold:
                add_segment(level_price, current_start, candle['datetime'], current_type, 'resistance')
                flip_markers.append({'level': level_price, 'time': candle['datetime'], 'new_type': 'resistance'})
                current_type = 'resistance'
                current_start = candle['datetime']

    add_segment(level_price, current_start, end_time, current_type, None)

level_segments_df = pd.DataFrame(level_segments) if level_segments else pd.DataFrame(columns=['level','start','flip_time','type','new_type'])
flip_markers_df = pd.DataFrame(flip_markers) if flip_markers else pd.DataFrame(columns=['level','time','new_type'])
level_segments_df.head(10)


,level,start,flip_time,type,new_type
0,61.5,2025-04-08 09:30:00,2025-04-08 11:30:00,support,resistance
1,61.5,2025-04-08 11:30:00,2025-04-09 19:30:00,resistance,support
2,61.5,2025-04-09 19:30:00,2025-04-10 05:00:00,support,resistance
3,61.5,2025-04-10 05:00:00,2025-04-14 06:00:00,resistance,support
4,61.5,2025-04-14 06:00:00,2025-04-14 12:30:00,support,resistance
5,61.5,2025-04-14 12:30:00,2025-04-15 01:30:00,resistance,support
6,61.5,2025-04-15 01:30:00,2025-04-15 06:00:00,support,resistance
7,61.5,2025-04-15 06:00:00,2025-04-16 05:00:00,resistance,support
8,61.5,2025-04-16 05:00:00,2025-04-29 04:00:00,support,resistance
9,61.5,2025-04-29 04:00:00,2025-05-12 02:30:00,resistance,support


In [8]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(
    x=df_levels['datetime'],
    open=df_levels['open'],
    high=df_levels['high'],
    low=df_levels['low'],
    close=df_levels['close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])

legend_shown = {'support': False, 'resistance': False}

if not level_segments:
    print('No level segments to display.')
else:
    for segment in level_segments:
        color = 'MediumSeaGreen' if segment['type'] == 'support' else 'Tomato'
        fig.add_trace(go.Scatter(
            x=[segment['start'], segment['flip_time']],
            y=[segment['level'], segment['level']],
            mode='lines',
            line=dict(color=color, width=2),
            name=f"{segment['type'].title()} level",
            legendgroup=segment['type'],
            showlegend=not legend_shown[segment['type']]
        ))
        legend_shown[segment['type']] = True

fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title='Levels with Gender Changes'
)

fig.show()


In [9]:
import plotly.graph_objects as go

MIN_REQUIRED_TOUCHES = 4
base_tolerance = bin_width / 2
shift_bars = 10

swing_data = df[['datetime', 'pivot', 'low', 'high', 'atr14']].copy()
swing_data['datetime'] = pd.to_datetime(swing_data['datetime'])
swing_data['tol'] = base_tolerance + swing_data['atr14'].fillna(0) * 0.5

datetime_index = df_levels['datetime'].reset_index(drop=True)

def shift_forward(ts, bars=shift_bars):
    if datetime_index.empty:
        return ts
    idx = datetime_index.searchsorted(ts)
    idx = min(idx + bars, len(datetime_index) - 1)
    return datetime_index.iloc[idx]

threshold_times = {}
for _, zone in zones.iterrows():
    level = zone['level']
    created_at = pd.to_datetime(zone['created'])
    swings = swing_data[swing_data['datetime'] >= created_at]
    if swings.empty:
        continue
    support_hits = swings[(swings['pivot'] == 1) & (np.abs(swings['low'] - level) <= swings['tol'])]
    resistance_hits = swings[(swings['pivot'] == 2) & (np.abs(swings['high'] - level) <= swings['tol'])]
    touches = pd.concat([support_hits[['datetime']], resistance_hits[['datetime']]]).sort_values('datetime')
    if len(touches) >= MIN_REQUIRED_TOUCHES:
        threshold_times[level] = touches.iloc[MIN_REQUIRED_TOUCHES - 1]['datetime']

segments_after_threshold = []
for segment in level_segments:
    threshold_time = threshold_times.get(segment['level'])
    if threshold_time is None:
        continue
    seg_start = pd.to_datetime(segment['start'])
    seg_end = pd.to_datetime(segment['flip_time'])
    adj_start = max(seg_start, shift_forward(threshold_time))
    if adj_start < seg_end:
        segments_after_threshold.append({
            'level': segment['level'],
            'type': segment['type'],
            'start': adj_start,
            'end': seg_end
        })

# Persist displayed segments for later cells.
displayed_segments = segments_after_threshold.copy()

fig = go.Figure(data=[go.Candlestick(
    x=df_levels['datetime'],
    open=df_levels['open'],
    high=df_levels['high'],
    low=df_levels['low'],
    close=df_levels['close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])

legend_shown = {'support': False, 'resistance': False}
if not segments_after_threshold:
    print('No levels reached the 4-touch threshold.')
else:
    for segment in segments_after_threshold:
        color = 'MediumSeaGreen' if segment['type'] == 'support' else 'Tomato'
        fig.add_trace(go.Scatter(
            x=[segment['start'], segment['end']],
            y=[segment['level'], segment['level']],
            mode='lines',
            line=dict(color=color, width=2),
            name=f"{segment['type'].title()} (>=4 combined touches)",
            legendgroup=segment['type'],
            showlegend=not legend_shown[segment['type']]
        ))
        legend_shown[segment['type']] = True

fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title='Levels plotted only after 4 combined touches (shifted 10 bars)'
)

fig.show()


In [10]:
from datetime import timedelta

recent_window = timedelta(days=60)
recent_touch_threshold = 4

swing_data = df[['datetime', 'pivot', 'low', 'high', 'atr14']].copy()
swing_data['datetime'] = pd.to_datetime(swing_data['datetime'])
swing_data['tol'] = bin_width / 2 + swing_data['atr14'].fillna(0) * 0.5

def count_recent_touches(level, start, end):
    window = swing_data[(swing_data['datetime'] >= start) & (swing_data['datetime'] <= end)]
    support_hits = window[(window['pivot'] == 1) & (np.abs(window['low'] - level) <= window['tol'])]
    resistance_hits = window[(window['pivot'] == 2) & (np.abs(window['high'] - level) <= window['tol'])]
    return len(support_hits) + len(resistance_hits)

# Only evaluate segments that were actually displayed in the previous chart.
segments_to_evaluate = displayed_segments if 'displayed_segments' in globals() else []

filtered_segments = []
expiry_markers = []
for seg in segments_to_evaluate:
    seg_end = pd.to_datetime(seg['end'])
    window_start = seg_end - recent_window
    touch_count = count_recent_touches(seg['level'], window_start, seg_end)

    if touch_count >= recent_touch_threshold:
        filtered_segments.append(seg)
    else:
        expiry_markers.append({'time': seg_end, 'level': seg['level']})

fig = go.Figure(data=[go.Candlestick(
    x=df_levels['datetime'],
    open=df_levels['open'],
    high=df_levels['high'],
    low=df_levels['low'],
    close=df_levels['close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])

legend_shown = {'support': False, 'resistance': False}
if not filtered_segments:
    print('No surviving segments after recent-touch filter.')
else:
    for seg in filtered_segments:
        color = 'MediumSeaGreen' if seg['type'] == 'support' else 'Tomato'
        fig.add_trace(go.Scatter(
            x=[seg['start'], seg['end']],
            y=[seg['level'], seg['level']],
            mode='lines',
            line=dict(color=color, width=2),
            name=f"{seg['type'].title()} (recent filter)",
            legendgroup=seg['type'],
            showlegend=not legend_shown[seg['type']]
        ))
        legend_shown[seg['type']] = True

if expiry_markers:
    fig.add_trace(go.Scatter(
        x=[m['time'] for m in expiry_markers],
        y=[m['level'] for m in expiry_markers],
        mode='markers',
        marker=dict(color='orange', size=9, symbol='x'),
        name='<4 touches in last 60 days'
    ))

fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title='Levels removed if fewer than 4 touches in past 60 days'
)


In [11]:
df_levels['ema100'] = df_levels['close'].ewm(span=100).mean()

# fig.add_trace(
#     go.Scatter(
#         x=df_levels['datetime'],
#         y=df_levels['ema20'],
#         mode='lines',
#         line=dict(color='yellow', width=2),
#         name='EMA 20'
#     )
# )
fig.add_trace(
    go.Scatter(
        x=df_levels['datetime'],
        y=df_levels['ema100'],
        mode='lines',
        line=dict(color='red', width=2),
        name='EMA 100'
    )
)
fig.show()